# ACCESS-NRI 2025 -- CMWG Workshop 

# Exporting ISSM Models

`pyISSM` allows ISSM models to be conveniently exported to regular gridded NetCDF using `pyissm.model.io.export_gridded_model()`. This function accepts an optional `variable_map` that defines variable names to be used for the exported fields. By default, this mapping uses the ISMIP6 variable naming convention.

Below we provide an example of exporting select model fields to a regular grid.

<div class="alert alert-block alert-info">
<b>AIM:</b> This notebook demonstrates the gridded export of an existing ISSM model.
</div>

In [ ]:
# Import required modules
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt


# Import pyISSM
# NOTE: Here, we assume that the `pyISSM` git repository has been cloned onto your machine. If `pyISSM` has been installed using `conda`, skip the `sys` calls.
import sys
import socket
hostname = socket.gethostname()
if 'gadi' in hostname:
    sys.path.append('/home/565/lb9857/gitRepos/pyISSM/src/')
else:
    sys.path.append('/Users/lawrence.bird/pyISSM/src/')
    import matplotlib
    matplotlib.use('MacOSX')
    %matplotlib inline
import pyissm

In [ ]:
# Load an existing ISSM model onto md

if 'gadi' in hostname:
    md = pyissm.model.io.load_model('/home/565/lb9857/gitRepos/CMWG_workshop_2025/sampleModels/Greenland.HistoricTransient_200yr.nc')
else:
    md = pyissm.model.io.load_model('/Users/lawrence.bird/access-dev/sampleModels/Greenland.HistoricTransient_200yr.nc')

## 1. Define a grid for interpolation

The first required step is to define a regular grid onto which the ISSM model is interpolated. This can be defined in a number of different ways. Below, we provide an efficient approach to generate a 5 km x 5 km grid.

In [ ]:
# Define the resolution of the grid
x_resolution = 5000
y_resolution = 5000

# Define a regular grid
grid_x = np.arange(md.mesh.x.min(), md.mesh.x.max(), x_resolution)
grid_y = np.arange(md.mesh.y.min(), md.mesh.y.max(), y_resolution)
grid_x, grid_y = np.meshgrid(grid_x, grid_y)

# Check the dimensions of the grid
print(f'Dimension of grid_x object: {grid_x.shape}')
print(f'Dimension of grid_y object: {grid_y.shape}')

# Calculate the grid size in km
print(f'Grid size in X direction (km): {(grid_x[0,-1] - grid_x[0, 0]) / 1000}')
print(f'Grid size in Y direction (km): {(grid_y[-1,0] - grid_y[0, 0]) / 1000}')

## 2. Define a variable map of desired variables to export (optional)

Once a regular grid has been defined, select model fields can be interpolated onto the grid and saved to NetCDF file using the `pyissm.model.io.export_gridded_model()` function. By default, `pyISSM` exports common input and output fields, as well as some ISMIP6-calculated fields.

To export only specific fields, users can supply a custom `variable_map` either as a `*.csv` file, or a `pd.Dataframe()`. For example, to export only the ice surface elevation and ice base elevation, the `variable_map` can be constructed as follows:

In [ ]:
# Define a variable map for exporting specific fields from the ISSM model
var_map = {
    'issmModelGroup': ['geometry', 'geometry'],
    'issmModelSubgroup': [np.nan, np.nan],
    'issmVariableName': ['surface', 'base'],
    'issmVariableUnit': ['m', 'm'],
    'outputVariableName': ['init_surface_altitude', 'init_base_altitude'],
    'outputVariableLongName': ['Initial ice surface altitude', 'Initial ice base altitude'],
    'outputVariableUnit': ['m', 'm']
}

# Convert the variable map to a pandas DataFrame
var_map = pd.DataFrame(var_map)

# View the variable map
var_map

## 3. Export gridded model

Once the grid and [optional] variable map have been defined, the model can be exported simply using `pyissm.model.io.export_gridded_model()`. This generates a NetCDF file that can be used for additional analysis.

In [ ]:
# Export the selecte model fields onto the defined grid
pyissm.model.io.export_gridded_model(md,
                                     out_file = 'Greenland.HistoricTransient_200yr_grid.nc',
                                     grid_x = grid_x,
                                     grid_y = grid_y,
                                     variable_map = var_map)

---
# Visualising the gridded model

Once the model has been exported, it can be visualised and analysed with other common Python modules, such as `Xarray`. Below, we load the gridded model using `xarray`, examine it, and plot it along-side the original ISSM model.

In [ ]:
# Load the gridded model
md_gridded = xr.open_dataset('Greenland.HistoricTransient_200yr_grid.nc')

# Examine the gridded model
md_gridded


In [ ]:
# Initialise a matplotlib figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5), sharex=True, sharey=True)

# Plot the ISSM model surface elevation
ax1 = pyissm.plot.plot_model_field(md,
                                   md.geometry.surface,
                                   ax = ax1,
                                   show_cbar = True,
                                   cbar_args = {'label': 'Surface elevation (m)'})

# Plot the gridded model surface elevation
md_gridded.init_surface_altitude.plot(ax = ax2,
                                      cbar_kwargs = {'label': 'Surface elevation (m)'}) 

# Set titles for the subplots
ax1.set_title('Surface elevation\n(ISSM Model mesh)', fontsize = 10)
ax2.set_title('Surface elevation\n(Gridded model)', fontsize = 10)

# Adjust the X/Y ticks (only need to adjust ax1 as all subplots share X/Y axes)
ax1.set_xticks([-5*1e5, 0, 5*1e5])
ax1.set_xticklabels(['-500', '0', '500'])
ax1.set_yticks([-1*1e6, -2*1e6, -3*1e6])
ax1.set_yticklabels(['-1000', '-2000', '-3000'])

# Adjust the axis labels
ax2.set_xlabel('X (km)')
ax2.set_ylabel('')

In [ ]:
print(f'The maximum ice surface elevation in the native ISSM model is: {np.round(md.geometry.surface.max(), 2)} m')
print(f'The maximum ice surface elevation in the gridded model is: {np.round(float(md_gridded.init_surface_altitude.max()), 2)} m')